# 0. Install Dependencies

In [50]:
# !pip install tensorflow gym keras-rl2 'gym[atari,accept-rom-license]'
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install pygame


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


# 1. Test Random Environment with OpenAI Gym

In [51]:
import gym
import time
import random
env = gym.make('SpaceInvaders-v0', render_mode = 'rgb_array')
height, width, channels = env.observation_space.shape
actions = env.action_space.n
episodes = 1
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0     
    while not done:        
        action = random.choice([0,1,2,3,4,5])
        state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    if done:
        observation = env.reset()
env.close()

Episode:1 Score:35.0


# 2. Create a Deep Learning Model with Keras

In [52]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam


In [53]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [54]:
model = build_model(height, width, channels, actions)

In [55]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               34603520  
_________________________________________________________________
dense_26 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_27 (Dense)             (None, 6)                

# 3. Build Agent with Keras-RL

In [56]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [57]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10)
    memory = SequentialMemory(limit=1, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1)
    return dqn

In [59]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3))
dqn.fit(env, nb_steps=1000, visualize=False, verbose=2)

Training for 1000 steps ...


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# dqn.fit(env, nb_steps=10, visualize=False, verbose=2)

In [ ]:
dqn = build_agent(model, actions)
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [ ]:
del model, dqn

In [ ]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')